### Setup

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import glob
import os
import numpy as np
from scipy import fftpack
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FastICA
from scipy.signal import butter, filtfilt
import altair as alt
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as scipy_signal
import pywt
from scipy.stats import wilcoxon
import matplotlib.cm as cm
from scipy.signal import welch
import random
import matplotlib.pyplot as plt



alt.data_transformers.disable_max_rows()
fs = 22050

In [2]:
# Which fish?
subjid = 'hydrolagusColliei_8'

loaded = np.load(f'{subjid}_data.npz', allow_pickle=True)
data = loaded['data']
freq_amp_table = loaded['freq_amp_table']
latency = loaded['latency'].item()
channel_keys = loaded['channel_keys'].tolist()
period_keys = ['prestim','stimresp']
metric_keys = ['mean','std']
submetric_keys = ['doub_freq_mag','SNR']
data

array({(np.float64(55.0), np.float64(115.0)): {'filename': 'data\\hydrolagusColliei_8_AdaptiveAEP_Ratfish_2506211524_1_run.mat', 'decision': array([[1.]]), 'period_len': 3528, 'ch1': array([[ 1.13919523,  0.29854812,  0.51171346, ...,  0.24535885,
         0.71333113,  0.17893058],
       [-0.53218433, -0.06263737,  0.94265154, ..., -0.704058  ,
         1.99663896,  3.66895166],
       [ 2.37567084,  1.16514833,  0.0586715 , ..., -1.92881097,
        -1.78592312, -0.78063422],
       ...,
       [ 0.52711035, -1.10291921, -1.87754631, ..., -0.50617291,
        -0.01370557, -0.80273809],
       [ 0.82851623,  0.15274419,  0.2325281 , ...,  1.65131694,
         1.86232438,  0.56490273],
       [ 2.03851384,  2.79068714,  2.93614113, ..., -1.96881957,
        -0.88677948, -1.29520543]], shape=(100, 10584)), 'ch2': array([[ 1.83689618,  1.50323847,  1.31520973, ..., -0.0951808 ,
        -0.09308123, -0.0600129 ],
       [-0.06205415,  0.44131932,  0.61360124, ...,  1.10729333,
         1.

### Which stimulus frequencies and amplitudes were tested?

In [3]:
freq_amp_table_df = pd.DataFrame(freq_amp_table).sort_values(by= [0,1],ascending=True)
print(list(data.item().keys()))
freq_amp_table_df

[(np.float64(55.0), np.float64(115.0)), (np.float64(55.0), np.float64(110.0)), (np.float64(55.0), np.float64(105.0)), (np.float64(55.0), np.float64(100.0)), (np.float64(55.0), np.float64(95.0)), (np.float64(220.0), np.float64(125.0)), (np.float64(220.0), np.float64(120.0)), (np.float64(220.0), np.float64(115.0)), (np.float64(220.0), np.float64(110.0)), (np.float64(220.0), np.float64(105.0)), (np.float64(220.0), np.float64(100.0)), (np.float64(220.0), np.float64(95.0)), (np.float64(220.0), np.float64(90.0)), (np.float64(880.0), np.float64(150.0)), (np.float64(880.0), np.float64(140.0)), (np.float64(100.0), np.float64(115.0)), (np.float64(100.0), np.float64(110.0)), (np.float64(100.0), np.float64(105.0)), (np.float64(100.0), np.float64(100.0)), (np.float64(440.0), np.float64(150.0)), (np.float64(1695.0), np.float64(150.0)), (np.float64(330.0), np.float64(150.0)), (np.float64(360.0), np.float64(150.0)), (np.float64(385.0), np.float64(150.0)), (np.float64(55.0), np.float64(90.0)), (np.floa

,0,1
30,55.0,90.0
4,55.0,95.0
29,55.0,95.0
3,55.0,100.0
28,55.0,100.0
2,55.0,105.0
27,55.0,105.0
1,55.0,110.0
26,55.0,110.0
0,55.0,115.0


### Artefact rejection

In [4]:
def remove_artefacts(data):
    for coord, dictionary in data.item().items():
        for channel in ['ch1', 'ch2', 'ch3', 'ch4']:
            rms_per_row = np.sqrt(np.mean(dictionary[channel]**2, axis=1))  # Get RMS of each row, axis = 1 means collapse rows
            rms_mean = np.mean(rms_per_row)
            rms_std = np.std(rms_per_row)
            
            # See which rows has an rms that is too high (i.e., artifact) and remove them
            threshold = rms_mean + (rms_std*3)
            keep_rows = rms_per_row <= threshold
            dictionary[channel] = dictionary[channel][keep_rows]
            dictionary[f'{channel}_total_trials'] = dictionary[channel].shape[0]
    
    for coord, dictionary in data.item().items():
        # Determine the minimum number of trials across all channels
        min_trials = min([dictionary[f'{channel}_total_trials'] for channel in ['ch1', 'ch2', 'ch3', 'ch4']])
        
        # Randomly select the same number of trials for each channel
        for channel in ['ch1', 'ch2', 'ch3', 'ch4']:
            # Set a random seed for reproducibility if needed
            np.random.seed(42)
            
            # Get indices for the channel
            channel_indices = np.arange(dictionary[channel].shape[0])
            
            # Randomly select indices
            selected_indices = np.random.choice(channel_indices, size=min_trials, replace=False)
            
            # Update the channel data with selected trials
            dictionary[channel] = dictionary[channel][selected_indices]
            
            # Update the total trials count
            dictionary[f'{channel}_total_trials'] = min_trials
    
    return data

data = remove_artefacts(data)

### Select stimulus

In [6]:
freq_amp_table_df

,0,1
30,55.0,90.0
4,55.0,95.0
29,55.0,95.0
3,55.0,100.0
28,55.0,100.0
2,55.0,105.0
27,55.0,105.0
1,55.0,110.0
26,55.0,110.0
0,55.0,115.0


In [6]:
# Select which stimulus frequency and amplitude to analyze
myfreq = 100
myamp = 115
specific_key = (np.float64(myfreq), np.float64(myamp))
current_cond = data.item()[specific_key]
current_cond

{'filename': 'data\\hydrolagusColliei_8_AdaptiveAEP_Ratfish_2506211524_1_run.mat',
 'decision': array([[1.]]),
 'period_len': 3528,
 'ch1': array([[-1.90478248, -0.51013877,  0.21818099, ..., -2.41159693,
         -2.18233484, -2.44676484],
        [-1.58028127, -1.89125188, -2.32329807, ...,  1.50300519,
          0.95915654,  0.26326357],
        [ 0.31464487,  1.88332015,  1.77560021, ...,  2.01815962,
          0.11296888,  0.54180739],
        ...,
        [ 0.20774143,  0.61126838,  0.08240838, ..., -0.78255883,
         -0.42236481, -0.52005344],
        [-0.26396343,  0.88724605,  1.26785262, ..., -2.68232566,
         -3.72301576, -3.28828677],
        [-1.58547189, -0.70884036, -0.21987232, ...,  1.940242  ,
          0.80577081, -0.96195598]], shape=(97, 10584)),
 'ch2': array([[ 0.62170794,  0.37623245,  0.09605563, ..., -0.30123091,
         -0.25264904, -0.26944565],
        [ 0.46412306,  0.9187397 ,  1.1891185 , ..., -0.13419793,
         -0.29574868, -0.36054395],
    

### Separate by periods

In [7]:
def separate_periods(current_cond, period_keys, channel_keys, latency):
    data = {
        'prestim': {},
        'stimresp': {}
    }
    
    period_len = current_cond['period_len']
    
    for period in period_keys:
        for channel in channel_keys:            
            if period == 'prestim':
                data[period][channel] = current_cond[channel][:,latency:latency+period_len]
                
            elif period == 'stimresp':
                data[period][channel] = current_cond[channel][:,latency+period_len:latency+period_len*2]
                
    return data

data = separate_periods(current_cond, period_keys, channel_keys, latency)

### Calculate Welch's fft

In [8]:
def compute_welch_fft(data, period_keys, channel_keys, fs):
    freq_step = 1
    pow_two = np.ceil(np.log2(fs/freq_step)) # Compute base 2 logarithm, i.e., what number i need to raise 2 by to get this value. But i wan a whole number so i ceil it
    n_fft = int(2**pow_two) # Nearest power of two for computational efficiency that gives me freq_step of 1
    
    fft_data = {
        'prestim': {},
        'stimresp': {}
    }
        
    for period in period_keys:
        for channel in channel_keys:
            fft_data[period][channel] = {}
            cur_chan = data[period][channel]
            trial_magnitudes = []
            for i_trial in range(cur_chan.shape[0]):
                cur_trial = cur_chan[i_trial]
                n_samples = len(cur_trial)
                
                f, Pxx = welch(cur_trial, fs=fs, nperseg=n_samples, nfft=n_fft, scaling='spectrum')
                
                if period == 'prestim' and channel == 'ch1' and i_trial == 1:
                    freq_vec = f
                
                magnitude = np.sqrt(Pxx)
                trial_magnitudes.append(magnitude)
                
            fft_data[period][channel] = np.vstack(trial_magnitudes)
                
    return fft_data, freq_vec

fft_data, freq_vec = compute_welch_fft(data, period_keys, channel_keys,fs)

### Plot FFTs from 3 random trials

In [ ]:
def plot_ffts(fft_data, period_keys, channel_keys, subjid, freq_vec, myfreq, myamp):
    total_trials = len(fft_data['stimresp']['ch1'])
    random_trials = random.sample(range(total_trials), 3)
    
    for random_trial in random_trials:
        # Create a new figure for each trial
        fig, axs = plt.subplots(len(channel_keys), len(period_keys), figsize=(10, 8), sharey=True)
        
        for i, channel in enumerate(channel_keys):
            for j, period in enumerate(period_keys):
                axs[i, j].plot(freq_vec, fft_data[period][channel][random_trial])
                axs[i, j].set_title(f'{channel} - {period}')
                axs[i, j].set_xlabel('Frequency (Hz)')
                axs[i, j].set_ylabel('Magnitude')
        
        plt.suptitle(f'{myfreq} Hz {myamp} dB: Welchs ffts for trial {random_trial}')
        plt.tight_layout()
        plt.savefig(f"{subjid}_rand_cum_avg_fft_trial_{random_trial}_{myfreq}Hz_{myamp}dB.png", 
                    format='png', 
                    bbox_inches='tight', 
                    dpi=300)
        plt.show(fig)  # Close the figure after saving